In [10]:
import pandas as pd
import geocoder
import seaborn as sn

In [11]:
filename1 = 'Data/311_Service_Requests_-_Vacant_and_Abandoned_Buildings_Reported'
filename2 = 'Data/311_Service_Requests_-_Graffiti_Removal'
filename3 = 'Data/311_Service_Requests_-_Pot_Holes_Reported'
filename4 = 'Data/311_Service_Requests_-_Sanitation_Code_Complaints'

In [12]:
df2 = pd.read_csv(filename2 + '.csv', encoding = "ISO-8859-1", nrows=100)

In [13]:
df2.head()

,Creation Date,Status,Completion Date,Service Request Number,Type of Service Request,What Type of Surface is the Graffiti on?,Where is the Graffiti located?,Street Address,ZIP Code,X Coordinate,Y Coordinate,Ward,Police District,Community Area,SSA,Latitude,Longitude,Location
0,01/01/2016,Completed,01/04/2016,16-00001120,Graffiti Removal,Brick - Unpainted,Front,4002 W BELDEN AVE,60639,1.149326e+06,1.914986e+06,35,25,20,NaN,41.922754,-87.726920,"(41.92275371369149, -87.72691963120627)"
1,01/01/2016,Completed,01/04/2016,16-00001753,Graffiti Removal,Brick - Unpainted,Alley,3324 W SCHUBERT AVE,60647,1.153530e+06,1.917756e+06,35,14,22,NaN,41.930270,-87.711830,"(41.93027022573564, -87.7118303754877)"
2,01/01/2016,Completed,01/04/2016,16-00002928,Graffiti Removal,Brick - Unpainted,Side,3700 S ARCHER AVE,60632,1.161132e+06,1.879956e+06,12,9,59,NaN,41.826349,-87.684472,"(41.82634906722627, -87.68447240074919)"
3,01/01/2016,Completed,01/04/2016,16-00003519,Graffiti Removal,Metal - Painted,Front,1503 W MORSE AVE,60626,1.165094e+06,1.946162e+06,49,24,1,24.0,42.007761,-87.668046,"(42.00776133839233, -87.66804558302428)"
4,01/01/2016,Completed,01/04/2016,16-00003521,Graffiti Removal,Metal - Painted,Front,1447 W MORSE AVE,60626,1.165368e+06,1.946171e+06,49,24,1,24.0,42.007765,-87.667774,"(42.00776549837537, -87.66777445467486)"


In [388]:
# finding out the worst neighborhoods
df2['ZIP Code'].value_counts().reset_index()

,index,ZIP Code
0,60647,15
1,60639,9
2,60609,9
3,60618,9
4,60632,8
5,60608,7
6,60617,5
7,60638,4
8,60626,4
9,60625,4


A value count of ZIP Code shows that the ZIP Code 60647 has the highest number of graffiti requests followed by 60639 and 60609.


In [141]:
#geocode it to know which neighborhoods are doing the worst

In [46]:
df2['ZIP Code'].value_counts()

60647    15
60639     9
60609     9
60618     9
60632     8
60608     7
60617     5
60638     4
60626     4
60625     4
60611     3
60612     3
60616     3
60660     2
60613     2
60623     2
60630     2
60641     2
60622     1
60614     1
60651     1
60629     1
60634     1
60607     1
60605     1
Name: ZIP Code, dtype: int64

In [348]:
#which type of surface has high number of requests
df_unsorted = df2['ZIP Code'].groupby(df2['What Type of Surface is the Graffiti on?']).count()

In [350]:
df_unsorted.sort_values(ascending=False).reset_index() # make the graph

,What Type of Surface is the Graffiti on?,ZIP Code
0,Brick - Unpainted,31
1,Metal,19
2,Brick - Painted,14
3,Aluminum Siding,12
4,Wood - Painted,9
5,"Cement (Sidewalk, Alley, Wall, Curb)",6
6,Other / Unknown Surface,4
7,Vinyl Siding,2
8,Wood - Unpainted,1
9,Marble/Granite,1


Brick - Unpainted followed by Metal and Brick - Painted are the largest types of graffiti requests that the City of Chicago has received.

In [225]:
# by neighborhoods and graffti types
d = df2.groupby(['What Type of Surface is the Graffiti on?', 'ZIP Code'])['Status'].count().reset_index()

In [237]:
d.sort(['Status', 'ZIP Code'], ascending=False)[:10] # analysis pending

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,What Type of Surface is the Graffiti on?,ZIP Code,Status
10,Brick - Painted,60609,5
48,Metal,60647,4
24,Brick - Unpainted,60625,4
7,Aluminum Siding,60647,3
15,Brick - Painted,60647,3
35,"Cement (Sidewalk, Alley, Wall, Curb)",60647,3
30,Brick - Unpainted,60639,3
25,Brick - Unpainted,60626,3
22,Brick - Unpainted,60618,3
16,Brick - Unpainted,60609,3


In [238]:
# caclulating response times

In [259]:
df2['Creation Date'] = df2['Creation Date'].apply(pd.to_datetime)

In [263]:
df2['Completion Date'] = df2['Completion Date'].apply(pd.to_datetime)

In [358]:
df2['Response Time'] = abs(df2['Creation Date'] - df2['Completion Date'])

In [363]:
df2['Response Time'] = df2['Response Time'].astype(pd.Timedelta).apply(lambda l: l.days)

In [368]:
response_time_grouped = df2['Response Time'].groupby(df2['ZIP Code']).mean().reset_index()

In [369]:
response_time_grouped.sort_values('Response Time', ascending=False)[:10]

,ZIP Code,Response Time
14,60626,12.250000
7,60614,9.000000
22,60647,7.333333
2,60608,7.285714
12,60623,6.500000
5,60612,6.000000
20,60639,4.444444
9,60617,4.400000
17,60632,4.375000
19,60638,4.000000


This table shows that the ZIP Codes of 60647 and 60608 have had requests with a very high mean response time. Finding those ZIP codes:

In [372]:
geocoder.google('60627', method='reverse'), geocoder.google('60614', method='reverse'), geocoder.google('60608', method='reverse')

(<[OK] Google - Reverse [13200 S Langley Ave, Chicago, IL 60827, USA]>,
 <[OK] Google - Reverse [1400 W Armitage Ave, Chicago, IL 60614, USA]>,
 <[OK] Google - Reverse [2251-2287 S Blue Island Ave, Chicago, IL 60608, USA]>)

In [371]:
response_time_grouped.sort_values('Response Time', ascending=True)[:10]

,ZIP Code,Response Time
23,60651,2.000000
11,60622,2.000000
21,60641,2.500000
3,60609,2.666667
0,60605,3.000000
16,60630,3.000000
15,60629,3.000000
13,60625,3.000000
24,60660,3.000000
6,60613,3.000000


In [373]:
geocoder.google('60651', method='reverse'), geocoder.google('60622', method='reverse')

(<[OK] Google - Reverse [1019 N Kedvale Ave, Chicago, IL 60651, USA]>,
 <[OK] Google - Reverse [1300 W Thomas St, Chicago, IL 60642, USA]>)

In [392]:
grouped_police_RT = df2[['Response Time', 'Latitude', 'Longitude']].groupby(df2['Police District']).mean().reset_index()

In [393]:
grouped_police_RT.sort_values('Response Time', ascending=False)

,Police District,Response Time,Latitude,Longitude
15,24,12.250000,42.015910,-87.671518
4,10,10.333333,41.848338,-87.711672
7,13,10.000000,41.884257,-87.693259
8,14,7.500000,41.919214,-87.702266
5,11,5.000000,41.874839,-87.690666
1,4,4.400000,41.712070,-87.545571
16,25,4.181818,41.920955,-87.755296
2,8,4.166667,41.801062,-87.736751
3,9,3.705882,41.828650,-87.660138
10,17,3.666667,41.961473,-87.720803


Police districts number 24, 10 and 13 have a high mean response time i.e. these police districts take a much higher time relatively to respond to grafiti requests

In [15]:
# lat, long of police district 24
geocoder.google(['42.015', '-87.67'], method='reverse')

<[OK] Google - Reverse [1542 W Sherwin Ave, Chicago, IL 60626, USA]>